## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [1]:
import requests
import pandas as pd
from datetime import datetime as dt
from bs4 import BeautifulSoup

KEYWORDS = ['python', 'парсинг']

html = BeautifulSoup(requests.get('https://habr.com/ru/all/').text, 'html.parser')
links = html.find_all("a", class_="post__title_link")

data = []
for link in links:
    post_html = BeautifulSoup(requests.get(link['href']).text, 'html.parser')
    post_text = post_html.find("div", {"class":"post__body_full"}).get_text()
    for keyword in KEYWORDS:
        if keyword in post_text.lower():
            date_string = post_html.find("span", class_="post__time")['data-time_published']
            date = dt.strptime(date_string, '%Y-%m-%dT%H:%MZ')
            title = post_html.find("span", class_="post__title-text").text
            data.append({
                "дата": date,
                "заголовок": title,
                "ссылка": link.href,
                "текст_статьи": post_text
            })
            break

pd.DataFrame(data)

,дата,заголовок,ссылка,текст_статьи
0,2021-04-05 11:19:00,Полезности для разработчика на Django,None,\nПредисловиеДля написания данной статьи был и...
1,2021-04-05 10:47:00,Magento 2 UI Components. Часть 1: общее устрой...,None,\nПривет! Меня зовут Павел и я Magento 2 бэкен...
2,2021-04-05 10:45:00,Impala для Python-разработчика на примере опре...,None,"\nВсем привет.Как известно, есть множество раз..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
import requests
import pandas as pd
from datetime import datetime as dt

EMAIL = ['kalinin.m.v@gmail.com', 'kalinin.m.v2@gmail.com']
API = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
HEADERS = {
    "Vaar-Version": "0",
    "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
    "Vaar-Header-App-Build-Version": "1.0.0"
}

breaches = [];

for item in EMAIL:
    r = requests.post(API, json={"emailAddresses": [item]}, headers=HEADERS)
    for breach in r.json()["breaches"].values():
        breaches.append({
            "почта": item,
            "дата утечки": dt.strptime(breach["publishDate"], '%Y-%m-%dT%H:%M:%SZ'),
            "источник утечки": breach["site"],
            "описание утечки": breach["description"]
        })
        
pd.DataFrame(breaches)

,почта,дата утечки,источник утечки,описание утечки
0,kalinin.m.v@gmail.com,2019-03-20,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
1,kalinin.m.v@gmail.com,2019-03-20,roll20.net,"In January 2019, Roll20 allegedly breached. Th..."
2,kalinin.m.v@gmail.com,2018-12-13,strongholdkingdoms.com,"In July of 2018, the online castle building MM..."
3,kalinin.m.v@gmail.com,2019-03-07,armorgames.com,"In December 2018, Armor Games' database was al..."
4,kalinin.m.v@gmail.com,2020-12-03,dark-wind.com,"In November 2020, a collection of over 23,000 ..."
5,kalinin.m.v@gmail.com,2018-10-04,mortalonline.com,"In June 2018, the online role playing game Mor..."
6,kalinin.m.v@gmail.com,2018-03-20,myfitnesspal.com,"In February 2018, MyFitnessPal's database was ..."
